# MAD! Multivariate Alteration Detection (Part 2)


In the previous part we introduced MAD. Mad can't be easily interpreted, therefore we introduce the iterative process.

In [2]:
# Enter your own export to assets path name here -----------
EXPORT_PATH = '../../exports/mad/assets/imad/'
# ------------------------------------------------


In [3]:
import ee

ee.Authenticate(auth_mode='notebook')
ee.Initialize(project='satvis-425514')


In [4]:
# Import other packages used in the tutorial
%matplotlib inline
import geemap
import numpy as np
import random, time
import matplotlib.pyplot as plt
from scipy.stats import norm, chi2

from pprint import pprint  # for pretty printing


### Previous Routines

In [5]:
%run mad_shared.ipynb